In [1]:
name = 'testrelu'
eps = 0.1
st_idx = 1
end_idx = 10

In [ ]:
# Model Training

# To fully train the model, change max_num_training_steps = 100000 in config.json
# A fully trained model will be more ReLU stable and hence much faster to verify.
!python train.py

In [ ]:
# Post process the model and convert it into a .mat file
# We specify the --do_eval flag to evaluate the accuracy and
# ReLU stability of the model before and after post-processing steps.
!python post_process_model.py --model_dir 'trained_models/relu_stable' --output name --do_eval 

In [ ]:
!julia --color=yes ./verification/verify_MNIST.jl {name} {eps} {st_idx} {end_idx} | wtee -a ./verification/logs/{name}.log

In [7]:
# Parse the result file
!python parser/parse_csv.py --csv_name ./verification/results/{name}__linf-norm-bounded-0.1__Inf__0.0/summary.csv

ProcessedSolveStatus                                ProvablyRobust     ...      Vulnerable
./verification/results/testrelu__linf-norm-boun...               8     ...               1

[1 rows x 3 columns]
                                                    BuildTime    ...      TotalTime
./verification/results/testrelu__linf-norm-boun...   8.553096    ...      23.724329

[1 rows x 3 columns]


In [9]:
# Parse the log file
!bash ./parser/convert_log.sh {name}
!python parser/parse_log.py --log_name {name}

For file verification/logs/testrelu.csv, it has 498.90000000000003 unstable relus on average
Layer 1: 311.1, Layer 2: 169.8, Layer 3: 18.0


In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.lite.constants.FLOAT16]
tflite_quant_model = converter.convert()


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import namedtuple
from datetime import datetime
import json
import math
import os
import sys
import time
import operator
from tensorflow.python import pywrap_tensorflow

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import scipy.io as sio

from pgd_attack import LinfPGDAttack
import models.MNIST_naive_ia
import models.MNIST_naive_ia_masked
from tqdm import trange

saved_model_dir = 'trained_models/relu_stable'
with open('config.json') as config_file:
    config = json.load(config_file)

model = models.MNIST_naive_ia.Model(config)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
tf.saved_model.save(model, "./trained_model")



ValueError: Expected a Trackable object for export, got <models.MNIST_naive_ia.Model object at 0x0000027A79CF87C8>.